# SQL-based attribution scoring

This notebook builds a Hive query that attributes conversions to model features

Inputs:

* JSON model spec
* DB location

In [1]:
from __future__ import print_function, division
%matplotlib inline
from hive_query import hive_query
import json

## Setup

In [8]:
model_json = 'Model_modeldate=1970-01-01,modeldim=bindp045.JSON'
stack = 'coeffstack_nwi_autotax3_mspnwi_nwi.coeffstack_base'
db = 'joao'
attr_stack = 'tmp_nwi_attr_stack'

## Extraction of model features

In [3]:
with open(model_json, 'r') as f:
    model = json.load(f)

all_features = model[u'ModelVarList']
all_features

[{u'AdditiveEventMetrics': [],
  u'Group': u'',
  u'IgnoreEventMetrics': [],
  u'ModelVarCoeff': -10.119515031874899,
  u'ModelVarIsDerived': u'0',
  u'ModelVarMktgFlag': u'0',
  u'ModelVarName': u'Intercept',
  u'NonAdditiveEventMetrics': [],
  u'TransDefn': u'0',
  u'coeffLBound': u'-100000',
  u'coeffUBound': u'100000',
  u'flagCompulsoryGroup': u'0',
  u'flagExclude': u'0',
  u'percContLBounds': u'0',
  u'percContUBounds': u'1',
  u'priorMean': u'0.1',
  u'rho': u'1E-7',
  u'varPriority': u'0'},
 {u'AdditiveEventMetrics': [u'stackmetric_count_event_all_w45'],
  u'Group': u'event_all',
  u'IgnoreEventMetrics': [],
  u'ModelVarCoeff': u'',
  u'ModelVarIsDerived': u'1',
  u'ModelVarMktgFlag': u'1',
  u'ModelVarName': u'stackmetric_count_event_all_w45',
  u'NonAdditiveEventMetrics': [],
  u'TransDefn': u'stackmetric_count_event_all_w45',
  u'coeffLBound': u'0',
  u'coeffUBound': u'100000',
  u'flagCompulsoryGroup': u'0',
  u'flagExclude': u'1',
  u'percContLBounds': u'0',
  u'percContU

In [14]:
features = [f for f in all_features if (len(str(f['ModelVarCoeff'])) > 0 and
                                        int(f['flagExclude']) == 0
                                       )]
features
mkt_features = [f for f in features if int(f['ModelVarMktgFlag']) == 1]
non_mkt_features = [f for f in features if int(f['ModelVarMktgFlag']) == 0]
feature_names = [f['ModelVarName'] for f in features]
feature_names

[u'Intercept',
 u'transvar_log_num_event_1_cl_w45',
 u'transvar_log_num_event_0_cl_w45',
 u'stackmetric_count_conversion_all_w60o30',
 u'transvar_log_num_webvisit_w60o30',
 u'transvar_dummy_event_organic',
 u'transvar_non_mktg_ind',
 u'transvar_log_num_event_1_im_w45',
 u'transvar_log_num_event_0_im_w45']

## Generation of transformed stack

In [37]:
skeleton = """
DROP VIEW IF EXISTS {db}.tmp_feature_stack;
DROP TABLE IF EXISTS {db}.tmp_feature_stack;
CREATE VIEW {db}.tmp_feature_stack AS
SELECT {features}
FROM {stack}
;"""

features_query = ', '.join(
    ['{} AS {}'.format(f['TransDefn'], f['ModelVarName'])
     if f['ModelVarName'] != 'Intercept' else
     '1 AS {}'.format(f['ModelVarName'])
     for f in features]
)

query = skeleton.format(
    db=db,
    stack=stack,
    features=features_query
)
print(query)
hive_query(query, raw=True, print_stderr=True)


DROP VIEW IF EXISTS joao.tmp_feature_stack;
DROP TABLE IF EXISTS joao.tmp_feature_stack;
CREATE VIEW joao.tmp_feature_stack AS
SELECT 1 AS Intercept, log(1+stackmetric_num_event_1_cl_w45) AS transvar_log_num_event_1_cl_w45, log(1+stackmetric_num_event_0_cl_w45) AS transvar_log_num_event_0_cl_w45, stackmetric_count_conversion_all_w60o30 AS stackmetric_count_conversion_all_w60o30, log(1+stackmetric_num_webvisit_w60o30) AS transvar_log_num_webvisit_w60o30, CASE WHEN stackmetric_num_event_0_org_w45 > 0 AND stackmetric_num_event_1_org_w45 > 0 THEN 1 ELSE 0 END AS transvar_dummy_event_organic, (1 - CASE WHEN  (stackmetric_count_event_all_w45)>0 THEN 1 ELSE 0 END) AS transvar_non_mktg_ind, log(1+stackmetric_num_event_1_im_w45) AS transvar_log_num_event_1_im_w45, log(1+stackmetric_num_event_0_im_w45) AS transvar_log_num_event_0_im_w45
FROM coeffstack_nwi_autotax3_mspnwi_nwi.coeffstack_base
;

Logging initialized using configuration in file:/etc/hive-1.2.1/hive-log4j.properties
Hive history fi

'intercept\ttransvar_log_num_event_1_cl_w45\ttransvar_log_num_event_0_cl_w45\tstackmetric_count_conversion_all_w60o30\ttransvar_log_num_webvisit_w60o30\ttransvar_dummy_event_organic\ttransvar_non_mktg_ind\ttransvar_log_num_event_1_im_w45\ttransvar_log_num_event_0_im_w45\n'

## Attribution

In [46]:
skeleton = """
DROP TABLE IF EXISTS {db}.tmp_attr_inter;
DROP VIEW IF EXISTS {db}.tmp_attr_inter;
CREATE VIEW {db}.tmp_attr_inter AS
SELECT
    {base_prob} AS base_prob,
    {total_prob} AS total_prob,
    ({base_prob})/({total_prob}) AS base_value,
    1 AS total_value,
    {raw_prob}
FROM {db}.tmp_feature_stack
;"""

all_terms = ' + '.join(
    ['{}*{}'.format(f['ModelVarCoeff'], f['ModelVarName']) for f in features])
mkt_terms = ' + '.join(
    ['{}*{}'.format(f['ModelVarCoeff'], f['ModelVarName']) for f in mkt_features])
nonmkt_terms = ' + '.join(
    ['{}*{}'.format(f['ModelVarCoeff'], f['ModelVarName']) for f in non_mkt_features])

base_prob = '1/(1+exp(-({})))'.format(nonmkt_terms)
total_prob = '1/(1+exp(-({})))'.format(all_terms)
raw_prob_list = []
for f in mkt_features:
    raw_prob_list.append(
        '1/(1+exp(-({}))) AS rawprob_{}'.format(
            ' + '.join(
                ['{0}*{1}'.format(fi['ModelVarCoeff'], fi['ModelVarName']) for fi in [f] + non_mkt_features]
            ),
            f['ModelVarName']
        )
    )
raw_prob = ', \n'.join(raw_prob_list)

query = skeleton.format(
    db=db,
    base_prob=base_prob,
    total_prob=total_prob,
    raw_prob=raw_prob
)
print(query)
hive_query(query, raw=True, print_stderr=True)


DROP TABLE IF EXISTS joao.tmp_attr_inter;
DROP VIEW IF EXISTS joao.tmp_attr_inter;
CREATE VIEW joao.tmp_attr_inter AS
SELECT
    1/(1+exp(-(-10.1195150319*Intercept + 1.49093563374*stackmetric_count_conversion_all_w60o30 + 0.0847728056169*transvar_log_num_webvisit_w60o30 + 4.09477209212*transvar_dummy_event_organic + 11.6100124983*transvar_non_mktg_ind))) AS base_prob,
    1/(1+exp(-(-10.1195150319*Intercept + 0.149208929329*transvar_log_num_event_1_cl_w45 + 0.546911058329*transvar_log_num_event_0_cl_w45 + 1.49093563374*stackmetric_count_conversion_all_w60o30 + 0.0847728056169*transvar_log_num_webvisit_w60o30 + 4.09477209212*transvar_dummy_event_organic + 11.6100124983*transvar_non_mktg_ind + 0.0427362557913*transvar_log_num_event_1_im_w45 + 0.184725181416*transvar_log_num_event_0_im_w45))) AS total_prob,
    (1/(1+exp(-(-10.1195150319*Intercept + 1.49093563374*stackmetric_count_conversion_all_w60o30 + 0.0847728056169*transvar_log_num_webvisit_w60o30 + 4.09477209212*transvar_dummy_eve

'base_prob\ttotal_prob\tbase_value\ttotal_value\trawprob_transvar_log_num_event_1_cl_w45\trawprob_transvar_log_num_event_0_cl_w45\trawprob_transvar_log_num_event_1_im_w45\trawprob_transvar_log_num_event_0_im_w45\n'

In [48]:
skeleton = """
DROP TABLE IF EXISTS {db}.tmp_attr;
DROP VIEW IF EXISTS {db}.tmp_attr;
CREATE TABLE {db}.tmp_attr AS
SELECT
    base_prob,
    total_prob,
    base_value,
    total_value,
    {mkt_features},
    {raw_prob_sum} AS raw_prob_sum,
    {rel_attr_pct},
    {mar_attr}
FROM {db}.tmp_attr_inter
;"""

rel_attr_pct = ',\n'.join(
    ['{0}/({1}) AS relattr_{0}'.format('rawprob_{}'.format(f['ModelVarName']),
                                       ' + '.join(['rawprob_{}'.format(f['ModelVarName']) for f in mkt_features])) for f in mkt_features]
)

mar_attr = ',\n'.join(
    ['(total_value - base_value) * {0}/({1}) AS marattr_{0}'.format('rawprob_{}'.format(f['ModelVarName']),
                                                                    ' + '.join(['rawprob_{}'.format(f['ModelVarName']) for f in mkt_features])) for f in mkt_features]
)

query = skeleton.format(
    db=db,
    mkt_features=', '.join(['rawprob_{}'.format(f['ModelVarName']) for f in mkt_features]),
    raw_prob_sum=' + '.join(['rawprob_{}'.format(f['ModelVarName']) for f in mkt_features]),
    rel_attr_pct=rel_attr_pct,
    mar_attr=mar_attr
)
print(query)
hive_query(query, raw=True, print_stderr=True)


DROP TABLE IF EXISTS joao.tmp_attr;
DROP VIEW IF EXISTS joao.tmp_attr;
CREATE TABLE joao.tmp_attr AS
SELECT
    base_prob,
    total_prob,
    base_value,
    total_value,
    rawprob_transvar_log_num_event_1_cl_w45, rawprob_transvar_log_num_event_0_cl_w45, rawprob_transvar_log_num_event_1_im_w45, rawprob_transvar_log_num_event_0_im_w45,
    rawprob_transvar_log_num_event_1_cl_w45 + rawprob_transvar_log_num_event_0_cl_w45 + rawprob_transvar_log_num_event_1_im_w45 + rawprob_transvar_log_num_event_0_im_w45 AS raw_prob_sum,
    rawprob_transvar_log_num_event_1_cl_w45/(rawprob_transvar_log_num_event_1_cl_w45 + rawprob_transvar_log_num_event_0_cl_w45 + rawprob_transvar_log_num_event_1_im_w45 + rawprob_transvar_log_num_event_0_im_w45) AS relattr_rawprob_transvar_log_num_event_1_cl_w45,
rawprob_transvar_log_num_event_0_cl_w45/(rawprob_transvar_log_num_event_1_cl_w45 + rawprob_transvar_log_num_event_0_cl_w45 + rawprob_transvar_log_num_event_1_im_w45 + rawprob_transvar_log_num_event_0_im_w45) 

'base_prob\ttotal_prob\tbase_value\ttotal_value\trawprob_transvar_log_num_event_1_cl_w45\trawprob_transvar_log_num_event_0_cl_w45\trawprob_transvar_log_num_event_1_im_w45\trawprob_transvar_log_num_event_0_im_w45\traw_prob_sum\trelattr_rawprob_transvar_log_num_event_1_cl_w45\trelattr_rawprob_transvar_log_num_event_0_cl_w45\trelattr_rawprob_transvar_log_num_event_1_im_w45\trelattr_rawprob_transvar_log_num_event_0_im_w45\tmarattr_rawprob_transvar_log_num_event_1_cl_w45\tmarattr_rawprob_transvar_log_num_event_0_cl_w45\tmarattr_rawprob_transvar_log_num_event_1_im_w45\tmarattr_rawprob_transvar_log_num_event_0_im_w45\n'

## Comparison with DN